# Weather info Scrap

In [178]:
from selenium import webdriver
import pandas as pd
import time

opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=opt)

## Getting the data

In [179]:
# first we access the website with the temperatures.
driver.get("https://www.weather-forecast.com/topographic-maps/pernambuco?symbols=cities.forecast.temp")
time.sleep(3)

In [180]:
# Python3 code to iterate over a list
list = [102, 136, 38, 54, 90, 10, 123, 107, 87, 84, 8] #id's of the desired cities

# Using for loop
df_temp=pd.DataFrame(columns=['local','hora','temp'])

for j in range(1,33):
    for i in list:
        aux_df=pd.DataFrame( [[i,
                              driver.find_element_by_xpath("/html/body/div[5]/div[2]/div[1]/div[2]/form/div[1]/div[1]/div[1]").text[-14:][:10],
                              driver.find_element_by_id(i).text]],
                             columns=['local','hora','temp'])
        df_temp=df_temp.append(aux_df,ignore_index=True)
    driver.find_element_by_xpath("//*[@id='interval_select']/option["+str(j)+"]").click()
    time.sleep(1)

In [181]:
df_temp

,local,hora,temp
0,102,26 Aug 9am,26
1,136,26 Aug 9am,26
2,38,26 Aug 9am,23
3,54,26 Aug 9am,22
4,90,26 Aug 9am,25
...,...,...,...
347,123,02 Sep 9pm,18
348,107,02 Sep 9pm,19
349,87,02 Sep 9pm,20
350,84,02 Sep 9pm,20


In [182]:
# Second we access the website with the weather forecast.
driver.get("https://www.weather-forecast.com/topographic-maps/pernambuco")
time.sleep(3)

In [183]:
df_weather=pd.DataFrame(columns=['local','hora','weather'])

for j in range(1,33):
    for i in list:
        aux_df=pd.DataFrame( [[i,
                              driver.find_element_by_xpath("/html/body/div[5]/div[2]/div[1]/div[2]/form/div[1]/div[1]/div[1]").text[-14:][:10],
                              driver.find_element_by_id(i).get_attribute("class")[15:]]],
                             columns=['local','hora','weather'])
        df_weather=df_weather.append(aux_df,ignore_index=True)
    driver.find_element_by_xpath("//*[@id='interval_select']/option["+str(j)+"]").click()
    time.sleep(1)

In [184]:
df_weather

,local,hora,weather
0,102,26 Aug 9am,dayrainshowers
1,136,26 Aug 9am,dayrainshowers
2,38,26 Aug 9am,dayrainshowers
3,54,26 Aug 9am,dayrainshowers
4,90,26 Aug 9am,daypartcloud
...,...,...,...
347,123,02 Sep 9pm,nightpartcloud
348,107,02 Sep 9pm,nightpartcloud
349,87,02 Sep 9pm,nightpartcloud
350,84,02 Sep 9pm,nightpartcloud


In [185]:
# Now let's merge the two dataframes.
final_df=pd.DataFrame()
final_df=pd.merge(df_weather,df_temp,on=['local','hora'])

In [186]:
final_df

,local,hora,weather,temp
0,102,26 Aug 9am,dayrainshowers,26
1,102,26 Aug 9am,dayrainshowers,26
2,102,26 Aug 9am,dayrainshowers,26
3,102,26 Aug 9am,dayrainshowers,26
4,136,26 Aug 9am,dayrainshowers,26
...,...,...,...,...
369,123,02 Sep 9pm,nightpartcloud,18
370,107,02 Sep 9pm,nightpartcloud,19
371,87,02 Sep 9pm,nightpartcloud,20
372,84,02 Sep 9pm,nightpartcloud,20


In [187]:
final_df=final_df.drop_duplicates()

In [188]:
final_df['hora'] = final_df['hora'].astype(str)

<ipython-input-188-25becbe8a037>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['hora'] = final_df['hora'].astype(str)


In [189]:
# This step helps the recognition of the datatype later on.
final_df2=final_df
final_df2['hora'] = final_df['hora'].str.slice(start=0, stop=7)+str(2021)+ " " + final_df['hora'].str.slice(start=7, stop=8)+":00 "+ final_df['hora'].str.slice(start=8, stop=11)

<ipython-input-189-307ec73e08f8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df2['hora'] = final_df['hora'].str.slice(start=0, stop=7)+str(2021)+ " " + final_df['hora'].str.slice(start=7, stop=8)+":00 "+ final_df['hora'].str.slice(start=8, stop=11)


In [190]:
final_df2

,local,hora,weather,temp
0,102,26 Aug 2021 9:00 am,dayrainshowers,26
4,136,26 Aug 2021 9:00 am,dayrainshowers,26
8,38,26 Aug 2021 9:00 am,dayrainshowers,23
12,54,26 Aug 2021 9:00 am,dayrainshowers,22
16,90,26 Aug 2021 9:00 am,daypartcloud,25
...,...,...,...,...
369,123,02 Sep 2021 9:00 pm,nightpartcloud,18
370,107,02 Sep 2021 9:00 pm,nightpartcloud,19
371,87,02 Sep 2021 9:00 pm,nightpartcloud,20
372,84,02 Sep 2021 9:00 pm,nightpartcloud,20


## Importing the data

In [120]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [176]:
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name(r'C:\Users\faria\Desktop\Data analyst 2021\Python\Reference\client_secret.json', scope)
client = gspread.authorize(creds)

sheet = client.open_by_key('17ACvlPPz5NSBl-0TMnVlExiTpeUUOAtq6kEoWsCcyN4').worksheet("Pernambuco")

In [177]:
sheet.update([final_df2.columns.values.tolist()] + final_df2.values.tolist())

{'spreadsheetId': '17ACvlPPz5NSBl-0TMnVlExiTpeUUOAtq6kEoWsCcyN4',
 'updatedRange': 'Pernambuco!A1:D331',
 'updatedRows': 331,
 'updatedColumns': 4,
 'updatedCells': 1324}